# Stacking Models – Logistic Regression + Random Forest

## 1. Motivation
- Complementary strengths:
  - Logistic regression → calibration, stability
  - Random forest → non-linear structure
- Goal: improve mid-probability region without sacrificing extremes

## 2. Temporal OOS Predictions
- Expanding window CV
- Strict out-of-sample predictions
- No metrics computed at this stage

## 3. Stacking Dataset Construction
- p_logit
- p_rf
- home_win

## 4. Meta-Model
- Logistic regression
- Interpretability preserved

## 5. Probabilistic Evaluation
- Log loss
- Brier score
- Bucket-based reliability

## 6. Comparison vs Base Models
- Where stacking helps
- Where it does not


In [1]:
features = [
    "home_form_weighted",
    "away_form_weighted",
    "home_momentum",
    "away_momentum",
    "home_advantage_diff"
]
target = "home_win"

import pandas as pd
from pathlib import Path
import sys
PROJECT_ROOT = Path().resolve().parents[0]
sys.path.append(str(PROJECT_ROOT))
DATA_DIR = PROJECT_ROOT / "data" / "processed"
df = pd.read_csv(DATA_DIR / "prematch_seasons18-24_home_advantage.csv")

In [2]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

C_VALUE = 10
MIN_TEST_SIZE = 50
seasons = (
    df["season"]
    .sort_values()
    .unique()
)

In [3]:
stack_rows = []

for i in range(1, len(seasons)):

    train_seasons = seasons[:i]
    test_season = seasons[i]

    train_df = df[df["season"].isin(train_seasons)].dropna(subset=features)
    test_df  = df[df["season"] == test_season].dropna(subset=features)

    if len(test_df) < MIN_TEST_SIZE:
        continue

    # ---- Logit base
    logit = LogisticRegression(
        C=C_VALUE,
        solver="lbfgs",
        max_iter=1000
    )
    logit.fit(train_df[features], train_df[target])
    p_logit = logit.predict_proba(test_df[features])[:, 1]

    # ---- RF base
    rf = RandomForestClassifier(
        n_estimators=200,
        max_depth=6,
        min_samples_leaf=0.03,
        max_features="sqrt",
        random_state=42,
        n_jobs=-1
    )
    rf.fit(train_df[features], train_df[target])
    p_rf = rf.predict_proba(test_df[features])[:, 1]

    # ---- Guardar OOS
    for j in range(len(test_df)):
        stack_rows.append({
            "season": test_season,
            "p_logit": p_logit[j],
            "p_rf": p_rf[j],
            "home_win": test_df[target].values[j]
        })

stack_df = pd.DataFrame(stack_rows)
stack_df


,season,p_logit,p_rf,home_win
0,2019 A,0.998216,0.971524,1
1,2019 A,0.048949,0.269544,0
2,2019 A,0.002067,0.027601,0
3,2019 A,0.109297,0.094335,0
4,2019 A,0.834656,0.621515,1
...,...,...,...,...
1692,2024 C,0.953577,0.837258,1
1693,2024 C,0.720950,0.719448,1
1694,2024 C,0.570385,0.579413,0
1695,2024 C,0.333303,0.535630,1


In [5]:
print(stack_df.shape)
print(stack_df.isna().sum())
stack_df[['p_logit', 'p_rf']].describe()

(1697, 4)
season      0
p_logit     0
p_rf        0
home_win    0
dtype: int64


,p_logit,p_rf
count,1697.000000,1697.000000
mean,0.450637,0.446685
std,0.420380,0.355280
min,0.000025,0.000000
25%,0.010580,0.066319
50%,0.348434,0.459090
75%,0.925617,0.795121
max,1.000000,0.997826


In [ ]:
X_stack = stack_df[['p_logit', 'p_rf']]
y_stack = stack_df['home_win']

meta_model = LogisticRegression(
    C=1.0,
    solver="lbfgs",
    max_iter=1000
)

meta_model.fit(X_stack, y_stack)


,feature,coef
0,p_logit,3.091349
1,p_rf,5.089741


In [17]:

stack_df['p_stack'] = meta_model.predict_proba(X_stack)[:, 1]
coef_meta = pd.DataFrame({
    "feature": X_stack.columns,
    "coef": meta_model.coef_[0]
})
print(coef_meta)
stack_df[['p_logit', 'p_rf', 'p_stack']].corr()

   feature      coef
0  p_logit  3.091349
1     p_rf  5.089741


,p_logit,p_rf,p_stack
p_logit,1.000000,0.963178,0.990419
p_rf,0.963178,1.000000,0.970757
p_stack,0.990419,0.970757,1.000000


In [9]:
from sklearn.metrics import log_loss, brier_score_loss

print(log_loss(stack_df['home_win'], stack_df['p_stack']))
print(brier_score_loss(stack_df['home_win'], stack_df['p_stack']))

0.22475704971598753
0.06770268228671172


In [10]:
BINS = np.arange(0, 1.01, 0.1)

stack_df["prob_bin"] = pd.cut(
    stack_df["p_stack"],
    bins=BINS,
    include_lowest=True
)
bucket_stack = (
    stack_df
    .groupby("prob_bin")
    .agg(
        n_samples=("home_win", "count"),
        avg_predicted_prob=("p_stack", "mean"),
        empirical_win_rate=("home_win", "mean")
    )
    .reset_index()
)

bucket_stack["brier_bucket"] = (
    bucket_stack["empirical_win_rate"] -
    bucket_stack["avg_predicted_prob"]
) ** 2
bucket_stack

C:\Users\USER\AppData\Local\Temp\ipykernel_15828\1405249920.py:10: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby("prob_bin")


,prob_bin,n_samples,avg_predicted_prob,empirical_win_rate,brier_bucket
0,"(-0.001, 0.1]",741,0.027926,0.013495,0.000208
1,"(0.1, 0.2]",75,0.141532,0.186667,0.002037
2,"(0.2, 0.3]",30,0.237589,0.200000,0.001413
3,"(0.3, 0.4]",29,0.351286,0.482759,0.017285
4,"(0.4, 0.5]",28,0.454285,0.750000,0.087448
5,"(0.5, 0.6]",40,0.547542,0.625000,0.006000
6,"(0.6, 0.7]",60,0.655131,0.666667,0.000133
7,"(0.7, 0.8]",72,0.758199,0.638889,0.014235
8,"(0.8, 0.9]",117,0.849656,0.837607,0.000145
9,"(0.9, 1.0]",505,0.957372,0.962376,0.000025


When base models rely on highly correlated signals, stacking provides limited marginal value and does not resolve mid-probability instability.